In [29]:
# Final_Project
# Douglas Schmieder
# Alan Flores
# Andrew Paek
# TODO (by Alan): Based on most of the sentiment analyzers we searched through, NLTK (Natural Language Toolkit) is a commonly used
package, they even have a neat how-to (http://www.nltk.org/howto/sentiment.html)

chatFileG = "testgood.txt"
chatFileB = "testbad.txt"


badWords = list()
goodWords = list()
chatLog = {}

posScore = 0
negScore = 0

def main():
    build_data()
    analyze_input()
    determine()
    
def determine():
    global posScore
    global negScore
    if posScore > negScore:
        print("Good person")
    else:
        print("Not good person")
    
    
def analyze_input():
    global posScore
    global negScore
    fp = open(chatFileB)
    num = 0
    for i in fp:
        tmpArray = i.split()
        for j in tmpArray:
            lj = j.lower()
            if lj in chatLog:
                chatLog[lj] += 1
            else:
                chatLog[lj] = 1

    for i in goodWords:
        if i in chatLog:
            posScore += chatLog[i]
    
    for i in badWords:
        if i in chatLog:
            negScore += chatLog[i]
            
    #print(posScore)
    #print(negScore)
    
def build_data():
    
    # Open Bad Words Text
    fp = open("badwords.txt")

    for i in fp:
        tmp = i.rstrip('\n')
        
        if tmp:
            badWords.append(tmp.lower())

    fp.close()
    
    
    # Open Good Words Text
    fp = open("goodwords.txt")
    
    for i in fp:
        tmp = i.rstrip('\n')
        if tmp:
            goodWords.append(tmp.lower())

    fp.close()
            
if __name__ == "__main__":
    main()



Not good person
